# Data Wrangling
Following notebook loops through a folder and reformats the Streeteasy data in a form usable by Python's Dash library.

## Melting Original Data
Original data has time series data where several columns contain the month that a specific value was captured. I need to convert the table in a dataframe where each observation has its own row. The melt function in Pandas automates this process 

In [2]:
import os
import pandas as pd

# Define directory with raw data and output directory for melted data
directory = r"C:\Users\Vaclav Barina\Documents\3. Study\2. Data Science\1. Fun Projects\2023_Streeteasy Housing Data\Cleaned Data\Unmelted"
output_directory = r"C:\Users\Vaclav Barina\Documents\3. Study\2. Data Science\1. Fun Projects\2023_Streeteasy Housing Data\Cleaned Data\Melted"

# Loop over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(directory, filename)) # Load file
        
        # Melt the DataFrame to convert the monthly columns into rows
        melted = pd.melt(df, id_vars=['areaName', 'Borough', 'areaType'], var_name='date', value_name='value')
        
        # Rename the 'value' column to the name of the current file
        file_name_without_extension = os.path.splitext(filename)[0]
        melted = melted.rename(columns={'value': file_name_without_extension})
        
        # Export the melted DataFrame to a CSV file in the output directory
        output_filename = os.path.splitext(filename)[0] + "_melted.csv"
        output_filepath = os.path.join(output_directory, output_filename)
        melted.to_csv(output_filepath, index=False)

## Combining Files

After melting all the data, I need to combine the melted dataframes into one giant dataframe to create the dashboard. The code below creates a new dataframe that can then be fed to the dashboard for visualization

In [8]:
import os
import pandas as pd

# Define the directory where your files are located
directory = r"C:\Users\Vaclav Barina\Documents\3. Study\2. Data Science\1. Fun Projects\2023_Streeteasy Housing Data\Cleaned Data\Melted"

# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Load the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(directory, filename))
        
        # Rename the monthly column to the name of the current file
        file_name_without_extension = os.path.splitext(filename)[0]
        df = df.rename(columns={'value': file_name_without_extension})
        
        # Merge the DataFrame into the merged data
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on=['areaName', 'Borough', 'areaType', 'date'])
        
# Save the merged data to a CSV file in a different directory
merged_directory = r"C:\Users\Vaclav Barina\Documents\3. Study\2. Data Science\1. Fun Projects\2023_Streeteasy Housing Data\Cleaned Data\Melted\Melted Master"
if not os.path.exists(merged_directory):
    os.makedirs(merged_directory)
merged_file = os.path.join(merged_directory, "merged_data.csv")
merged_df.to_csv(merged_file, index=False)
